In [1]:
from array import array
import math

In [2]:
def load_data(file):
    return tuple(array('d', map(float, line.split(','))) for line in open(file, 'r'))
def load_labels(file):
    return array('b', map(lambda i : int(i) - 1, open(file, 'r')))
def load_dataset():
    return load_data('data.csv'), load_labels('data-labels.csv')
def load_weights():
    return load_data('theta1.csv'), load_data('theta2.csv')

In [3]:
data, labels = load_dataset()
theta1, theta2 = load_weights()

In [4]:
def dot(left, right):
    return math.fsum(l * r for l, r in zip(left, right))
def sigmoid(X):
    return 1 / (1 + math.exp(-X))
def neuron_forward(data, weights, activate=sigmoid):
    return activate(dot(data, weights[1:]) + weights[0])
def layer_forward(data, weights, activate=sigmoid):
    activations = array('d', (0,) * len(weights))
    for neuron_index, neuron_weights in enumerate(weights):
        activations[neuron_index] = neuron_forward(data, neuron_weights, activate)
    return activations
def network_forward(data, theta1, theta2, activate=sigmoid):
    activations = layer_forward(data, theta1, activate)
#     score = array('d', map(lambda neuron_weights : dot(activations, neuron_weights), theta2))
    score = layer_forward(activations, theta2, activate)
    return score
def predict(data, theta1, theta2, activate=sigmoid):
    score = network_forward(data, theta1, theta2, activate=sigmoid)
    return score.index(max(score))
def batch_predict(batch, theta1, theta2, activate=sigmoid):
    return array('b', map(lambda data : predict(data, theta1, theta2, activate), batch))
def n_errors(prediction, labels):
    return sum((prediction - label) != 0 for prediction, label in zip(predictions, labels))
def error_rate(prediction, labels):
    return n_errors(prediction, labels) / len(labels)

In [5]:
predictions = batch_predict(data, theta1, theta2)

In [6]:
error_rate(predictions, labels)

0.0248